In [23]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
import requests
from urllib.request import Request, urlopen
from bs4.element import Comment
from urllib.error import HTTPError
import re

In [2]:
sites = ['https://www.drawdown.org/', 'https://www.c40.org/',  'https://socialcapitalmarkets.net/',  'https://www.microsoft.com/en-us/ai/ai-for-earth-grants', 
         'https://youngstartup.com/west2019/?utm_source=IPs&utm_campaign=ClimateLink', 'https://www.acterra.org/',
         'https://www.meetup.com/ClimateLink-SanFrancisco/events/260257852/', 'https://meetingoftheminds.org/',
        'https://wearesocial.com/global-digital-report-2019',
'https://www.statista.com/statistics/272014/global-social-networks-ranked-by-number-of-users/',
'https://www.statista.com/chart/17613/most-popular-websites/',
'http://digg.com/2019/most-visited-websites-world',
'https://www.weforum.org/agenda/2017/03/most-popular-social-networks-mapped/',
'https://vincos.it/world-map-of-social-networks/',
'https://www.pewinternet.org/fact-sheet/social-media/',
'https://knowledge.wharton.upenn.edu/article/impact-of-social-media/',
'https://www.fastcompany.com/90281575/social-media-predictions-for-2019-a-return-to-personal-authenticity',
'https://www.knightfoundation.org/funding-opp-research-norms-rules-governance-internet-digital-platforms',
'https://www.forbes.com/sites/ryanholmes/2018/10/29/are-facebook-groups-the-future-of-social-media-or-a-dead-end/',
'https://blog.hootsuite.com/social-media-trends/']

In [3]:
links = open("Links.txt","r") #download google doc as .txt file to get Links.txt
with open('Links.txt', 'r', encoding="utf8") as file:
    data = file.read().replace('\n', ' ')
def Find(data):  
    return re.findall(r'(https?://[^\s]+)', data)
sites = sites + Find(data)

In [4]:
headers = {'User-Agent':'Chrome/78.0'} #change this based on your browser. 
#check here https://www.whatismybrowser.com/detect/what-is-my-user-agent
filter_words = ['http', 'www']
html_corpus = [] 
scraped = []
errors = 0
for base in sites:
    #this section before the next for loop gets html from base site before looping over all links on base site
    req = Request(base, headers=headers)
    try: #error handling to catch dead links, etc., and prevent code from stopping for that   
        page = urlopen(req)
        soup = BeautifulSoup(page, 'html.parser')
        html_corpus.append(soup.prettify())
        scraped.append(base)
    except Exception as e:
        #print('error ' + base) uncomment this to help with debugging
        errors = errors + 1
        #continue
    for link in soup.findAll('a'): #iterate over all links on base site
        if link.has_attr('href') and not any(x in link['href'] for x in filter_words) and len(link['href']) > 0:
            #line above prevents scraping external link
            if base[len(base) - 1] == '/' and link['href'][0] == '/': #double slash between base url and href 
                #can lead to invalid urls so I remove extra one here 
                site = base + link['href'][1:]
            else:
                site = base + link['href']
            if site not in scraped: #makes sure site isn't scraped twice
                #print(site) uncomment this to help with debugging
                linkreq = Request(site,headers=headers)
                try:
                    linkpage = urlopen(linkreq)
                    linksoup = BeautifulSoup(linkpage, 'html.parser')
                    html_corpus.append(linksoup.prettify())
                    scraped.append(site)
                except Exception as e:
                    #print('error ' + site) uncomment this to help with debugging
                    errors = errors + 1
                    #continue

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

In [8]:
data = pd.DataFrame(list(zip(scraped, html_corpus)), 
               columns =['Site', 'HTML']) 
data.to_csv('html.csv', index = False)

## To get text only

In [ ]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True
def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

In [ ]:
headers = {'User-Agent':'Chrome/78.0'} #change this based on your browser. 
#check here https://www.whatismybrowser.com/detect/what-is-my-user-agent
filter_words = ['http', 'www']
scraped  = []
text_corpus = [] #this will be the same length as the sites list
errors = 0
for base in sites:
    #print("BASE " + base) uncomment this to help with debugging
    #this section before the next for loop gets html from base site before looping over all links on base site
    try: #error handling to catch dead links, etc., and prevent code from stopping for that
        html = urllib.request.urlopen(base).read()
        soup = BeautifulSoup(html, 'html.parser')
        text_corpus.append(text_from_html(html))
        scraped.append(base)
    except Exception as e:
        #print('error ' + base) uncomment this to help with debugging
        errors = errors + 1
    for link in soup.findAll('a'): #iterate over all links on base site
        if link.has_attr('href') and not any(x in link['href'] for x in filter_words) and len(link['href']) > 0:
            #line above prevents scraping external link
            if base[len(base) - 1] == '/' and link['href'][0] == '/': #double slash between base url and href 
                #can lead to invalid urls so I remove extra one here 
                site = base + link['href'][1:]
            else:
                site = base + link['href']
            if site not in scraped: #makes sure site isn't scraped twice
                #print(site) uncomment this to help with debugging
                try:
                    html_link = urllib.request.urlopen(site).read()
                    text_corpus.append(text_from_html(html_link))
                    scraped.append(site)
                except Exception as e:
                    #print('error ' + site) uncomment this to help with debugging
                    errors = errors + 1